In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Time
import time

# Random and os for reproducibility
import random
import os

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
import tensorflow as tf

# Processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Data imputation
from sklearn.impute import SimpleImputer

# Making keras compatible with scikit learn api
# https://scikit-learn.org/stable/developers/develop.html
from sklearn.base import BaseEstimator, RegressorMixin

# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

# Converting Times Series Data to 3D format
from src.utils.split_sequences import split_sequences

2024-07-15 22:37:06.943301: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-15 22:37:06.945593: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-15 22:37:06.991493: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-15 22:37:06.992622: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 22:37:07.917893: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

<IPython.core.display.Javascript object>

# Functions and definitions

### Convert train/test data to 3D format

In [3]:
def generate_sequences_helper(
    dataset, cement_types, dates=None, timesteps=None, split_by_cement_type=False
):
    index_train = dataset["y_train"].index
    index_test = dataset["y_test"].index

    dataset["y_train"] = dataset["y_train"].reset_index(drop=True)
    dataset["y_test"] = dataset["y_test"].reset_index(drop=True)

    if dates is not None:
        dataset["dates_train"] = dates[index_train].reset_index(drop=True)
        dataset["dates_test"] = dates[index_test].reset_index(drop=True)

    dataset["cement_types_train"] = cement_types.loc[index_train].reset_index(drop=True)
    dataset["cement_types_test"] = cement_types.loc[index_test].reset_index(drop=True)

    dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

    return dataset

<IPython.core.display.Javascript object>

In [4]:
def generate_sequences(
    dataset,
    timesteps,
    split_by_cement_type=False,
    train_columns=[],
    test_columns=[],
):
    if split_by_cement_type:
        dataset["x_train"], dataset["y_train"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_train"],
                    pd.DataFrame(dataset["x_train"], columns=train_columns),
                    dataset["cement_types_train"],
                    dataset["y_train"],
                ],
                axis=1,
            ),
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_test"],
                    pd.DataFrame(dataset["x_test"], columns=test_columns),
                    dataset["cement_types_test"],
                    dataset["y_test"],
                ],
                axis=1,
            ),
            timesteps,
        )
    else:
        dataset["x_train"], dataset["y_train"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_train"], columns=train_columns),
                    dataset["y_train"],
                ],
                axis=1,
            ).values,
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_test"], columns=test_columns),
                    dataset["y_test"],
                ],
                axis=1,
            ).values,
            timesteps,
        )
    return dataset

<IPython.core.display.Javascript object>

### Data preprocessing

In [5]:
def impute_data(dataset, imputer=None, imputer_params=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply imputation to the data
    if imputer is not None:
        imputer = imputer() if imputer_params is None else imputer(**imputer_params)
        x_train = imputer.fit_transform(x_train)
        x_test = imputer.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [6]:
def transform_data(dataset, transformer=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply data normalization/standardization to the data
    if transformer is not None:
        scaler = transformer()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [7]:
def preprocess_data(dataset, transformer=None, imputer=None, imputer_params=None):
    dataset = impute_data(dataset, imputer, imputer_params)
    dataset = transform_data(dataset, transformer)
    return dataset

<IPython.core.display.Javascript object>

### Train and evaluate the model

In [8]:
def train_and_evaluate_model(Estimator, dataset, estimator_params=None):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Instantiate the model
    model = Estimator() if estimator_params is None else Estimator(estimator_params)

    # Fitting the model
    model.fit(x_train, y_train)

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return model, score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

In [9]:
def evaluate_model(model, dataset):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

### Custom Cross Validate

In [10]:
def custom_cross_validate(
    Estimator,
    Imputer,
    Transform,
    x,
    y,
    cv,
    timesteps,
    dates=None,
    cement_types=None,
    estimator_params=None,
    imputer_params=None,
    split_by_cement_type=True,
):
    results = []
    scores = []

    for train_index, test_index in cv.split(x):
        dataset = {
            "dates_train": dates[train_index].reset_index(drop=True),
            "cement_types_train": cement_types.loc[train_index].reset_index(drop=True),
            "x_train": x.loc[train_index].reset_index(drop=True),
            "y_train": y[train_index].reset_index(drop=True),
            "dates_test": dates[test_index].reset_index(drop=True),
            "cement_types_test": cement_types.loc[test_index].reset_index(drop=True),
            "x_test": x.loc[test_index].reset_index(drop=True),
            "y_test": y[test_index].reset_index(drop=True),
        }

        # Preprocess the dataset
        dataset = preprocess_data(dataset, Transform, Imputer, imputer_params)

        # generate sequences (3D format)
        dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

        # Train and Evaluate the model
        score = train_and_evaluate_model(Estimator, dataset, estimator_params)
        scores.append(score)

    # After every iteration metrics results are appended together
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += [value]
    results.append(scores_final)
    return results

<IPython.core.display.Javascript object>

## Model Definition

In [11]:
class BidirectionalLSTM1(BaseEstimator, RegressorMixin):
    def __init__(self, params):
        self.model = self.get_model()
        self.batch_size = 64
        self.epochs = 300
        self.verbose = params.get("verbose", 0)
        self.callbacks = params.get("callbacks", None)
        self.validation_split = params.get("validation_split", None)

    def fit(self, X=None, y=None):
        self.history = self.model.fit(
            X,
            y,
            batch_size=self.batch_size,
            epochs=self.epochs,
            verbose=self.verbose,
            callbacks=self.callbacks,
            validation_split=self.validation_split,
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(
                    units=128, activation="relu", return_sequences=True
                )
            )
        )
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=64, activation="relu")
            )
        )
        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [12]:
class BidirectionalLSTM2(BaseEstimator, RegressorMixin):
    def __init__(self, params):
        self.model = self.get_model()
        self.batch_size = 64
        self.epochs = 300
        self.verbose = params.get("verbose", 0)
        self.callbacks = params.get("callbacks", None)
        self.validation_split = params.get("validation_split", None)

    def fit(self, X=None, y=None):
        self.history = self.model.fit(
            X,
            y,
            batch_size=self.batch_size,
            epochs=self.epochs,
            verbose=self.verbose,
            callbacks=self.callbacks,
            validation_split=self.validation_split,
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(
                    units=128, activation="relu", return_sequences=True
                )
            )
        )
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=64, activation="relu", return_sequences=True)
            )
        )
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=32, activation="relu")
            )
        )

        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [13]:
class BidirectionalLSTM3(BaseEstimator, RegressorMixin):
    def __init__(self, params):
        self.model = self.get_model()
        self.batch_size = 64
        self.epochs = 300
        self.verbose = params.get("verbose", 0)
        self.callbacks = params.get("callbacks", None)
        self.validation_split = params.get("validation_split", None)

    def fit(self, X=None, y=None):
        self.history = self.model.fit(
            X,
            y,
            batch_size=self.batch_size,
            epochs=self.epochs,
            verbose=self.verbose,
            callbacks=self.callbacks,
            validation_split=self.validation_split,
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=128, activation="elu", return_sequences=True)
            )
        )
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=64, activation="elu", return_sequences=True)
            )
        )
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=32, activation="elu")
            )
        )

        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [14]:
class BidirectionalLSTM4(BaseEstimator, RegressorMixin):
    def __init__(self, params):
        self.model = self.get_model()
        self.batch_size = 64
        self.epochs = 300
        self.verbose = params.get("verbose", 0)
        self.callbacks = params.get("callbacks", None)
        self.validation_split = params.get("validation_split", None)

    def fit(self, X=None, y=None):
        self.history = self.model.fit(
            X,
            y,
            batch_size=self.batch_size,
            epochs=self.epochs,
            verbose=self.verbose,
            callbacks=self.callbacks,
            validation_split=self.validation_split,
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(
                    units=128, activation="relu", return_sequences=True
                )
            )
        )
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=64, activation="relu", return_sequences=True)
            )
        )
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=32, activation="relu", return_sequences=True)
            )
        )
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=16, activation="relu")
            )
        )

        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [15]:
class BidirectionalLSTM5(BaseEstimator, RegressorMixin):
    def __init__(self, params):
        self.model = self.get_model()
        self.batch_size = 64
        self.epochs = 300
        self.verbose = params.get("verbose", 0)
        self.callbacks = params.get("callbacks", None)
        self.validation_split = params.get("validation_split", None)

    def fit(self, X=None, y=None):
        self.history = self.model.fit(
            X,
            y,
            batch_size=self.batch_size,
            epochs=self.epochs,
            verbose=self.verbose,
            callbacks=self.callbacks,
            validation_split=self.validation_split,
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=128, activation="elu", return_sequences=True)
            )
        )
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(units=64, activation="elu")
            )
        )
        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [16]:
def pad_time_series(dataframe, timesteps):
    """
    Pad timeseries with zeros
    """
    df_tmp = pd.DataFrame(
        dict(
            zip(
                dataframe.columns,
                [[0 for _ in range(timesteps - 1)] for _ in range(dataframe.shape[1])],
            )
        )
    )
    df_tmp[DATE] = dataframe[DATE].iloc[0]
    return pd.concat([df_tmp, dataframe], axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

In [17]:
def split_sequences_per_cement_type(dataframe, timesteps, pad=False):
    """
    Create sequences per cement time
    to avoid having parts of the sequence
    of different types of cement.
    """
    if timesteps == 1:
        return split_sequences(
            dataframe.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps
        )

    dates = dataframe[DATE][timesteps - 1 :]
    data = []
    dataframes = []

    for cement_type in CEMENT_TYPES:
        data.append(dataframe[dataframe[cement_type] == 1])
    data.append(dataframe[(dataframe[CEMENT_TYPES] == 0).all(axis=1)])

    for df in data:
        if pad:
            dates = df[DATE].reset_index(drop=True)
            df = pad_time_series(df, timesteps).reset_index(drop=True)
        else:
            dates = df[DATE][timesteps - 1 :].reset_index(drop=True)
        x, y = split_sequences(df.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps)
        x = pd.DataFrame({"Sequences": [sample.tolist() for sample in x]})
        y = pd.DataFrame({"Target": y})
        dataframes.append(pd.concat([dates, x, y], axis=1))

    data = pd.concat(dataframes, axis=0)
    data[DATE] = pd.to_datetime(data[DATE])
    data = data.sort_values(by=DATE).reset_index(drop=True)
    x = data["Sequences"]
    y = data["Target"].values
    x = np.array(x.tolist())

    return x, y

<IPython.core.display.Javascript object>

# Settings for Reproducibility

In [18]:
def set_seeds():
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    os.environ["PYTHONHASHSEED"] = str(SEED)
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)


# https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed

<IPython.core.display.Javascript object>

In [19]:
def set_global_determinism():
    set_seeds()

    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

<IPython.core.display.Javascript object>

In [20]:
index_to_save = 10
model_index = 1

<IPython.core.display.Javascript object>

In [21]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}
DATE = "Date"
CEMENT_TYPES = [
    "Cement_Type_CEM B",
    "Cement_Type_CEM C",
    "Cement_Type_CP II-F-32",
    "Cement_Type_CP II-F-40",
    "Cement_Type_CP V-ARI",
    "Cement_Type_Type I-II",
    "Cement_Type_Type III",
    "Cement_Type_Type IL",
]

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [22]:
results_to_save = []

results_dict = {
    "Category": "Global Model",
    "Company": "INN",
    "Plant": "INN",
    "Features": "Chemical + Mineralogical + Properties CS Less",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "BidirectionalLSTM",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [23]:
df = pd.read_csv("../../../../../../../data/processed/inn/global_dataset_inn.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we use all available features

In [24]:
df_copy = df.copy()
df_copy = pd.get_dummies(data=df_copy, columns=["Cement_Type"], drop_first=True)

df_copy = df_copy.drop(
    [
        "CS1",
        "CS3",
        "CS7",
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

In [25]:
df_copy[CEMENT_TYPES] = df_copy[CEMENT_TYPES].astype(int)
dates = df["Date"].copy()
x = df_copy.drop(["Date", "CS28", "Factory_Plant"] + CEMENT_TYPES, axis=1)
y = df_copy["CS28"]
train_columns = x.columns

<IPython.core.display.Javascript object>

In [26]:
df_copy["Factory_Plant"].value_counts()

Factory_Plant
partner_i    1223
partner_ii    1051
partner_iv          906
Name: count, dtype: int64

<IPython.core.display.Javascript object>

<h2>1. Dataset: df_copy</h2> <br>In this dataset all features are used.

In [27]:
def prepare_dataset(
    dataframe_original=df,
    dataframe_copy=df_copy,
    train_size=0.8,
    test_size=0.2,
    ignore_test_set=False,
    timesteps=1,
    split_by_cement_type=True,
):
    dataframe_original = df.copy()
    dataframe_copy = df_copy.copy()
    dataframe_copy[CEMENT_TYPES] = dataframe_copy[CEMENT_TYPES].astype(int).copy()
    dates = dataframe_original["Date"].copy()
    x = dataframe_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
    y = dataframe_copy["CS28"]
    cement_types = dataframe_copy[CEMENT_TYPES].copy()

    # Split the dataframe by plant
    plants = dataframe_original["Factory_Plant"].unique()
    train_indexes = []
    test_indexes = []
    plants_test_indexes = {}

    for plant in plants:
        plant_df = dataframe_original[dataframe_original["Factory_Plant"] == plant]
        plant_df = plant_df.sort_values(by="Date")
        plant_indices = plant_df.index
        train_end_index = int(len(plant_indices) * train_size)
        # plants_indexes[]

        if not ignore_test_set:
            train_indexes.extend(plant_indices[:train_end_index])
            test_indexes.extend(plant_indices[train_end_index:])
            plants_test_indexes[plant] = plant_indices[train_end_index:]

        else:
            train_indexes.extend(plant_indices[:train_end_index])
            test_indexes.extend(plant_indices[train_end_index:])
            plants_test_indexes[plant] = plant_indices[train_end_index:]

    train_index = dataframe_original.loc[train_indexes].sort_values(by="Date").index
    test_index = dataframe_original.loc[test_indexes].sort_values(by="Date").index

    plant_id_series = x["Factory_Plant"].copy()
    x = x.drop(["Factory_Plant"], axis=1)

    dataset = {
        "dates_train": dates[train_index].reset_index(drop=True).copy(),
        "cement_types_train": cement_types.loc[train_index]
        .reset_index(drop=True)
        .copy(),
        "x_train": x.loc[train_index].reset_index(drop=True).copy(),
        "y_train": y[train_index].reset_index(drop=True).copy(),
        "dates_test": dates[test_index].reset_index(drop=True).copy(),
        "cement_types_test": cement_types.loc[test_index].reset_index(drop=True).copy(),
        "x_test": x.loc[test_index].reset_index(drop=True).copy(),
        "y_test": y[test_index].reset_index(drop=True).copy(),
    }

    # Preprocess the dataset
    dataset = preprocess_data(dataset, None, SimpleImputer, {"strategy": "median"})
    # return dataset, plant_id_series.loc[test_index].reset_index(drop=True).copy()
    dataset["x_test"] = np.concatenate(
        [
            dataset["x_test"],
            plant_id_series.loc[test_index]
            .reset_index(drop=True)
            .copy()
            .values.reshape(-1, 1),
        ],
        axis=1,
    )
    index_list = x.columns.tolist()
    index_list.append("Factory_Plant")
    test_columns = pd.Index(index_list)

    # generate sequences (3D format)
    dataset = generate_sequences(
        dataset,
        timesteps=timesteps,
        split_by_cement_type=split_by_cement_type,
        train_columns=train_columns,
        test_columns=test_columns,
    )

    return dataset

<IPython.core.display.Javascript object>

In [28]:
def prepare_dataset_helper(timesteps):
    dataset = prepare_dataset(timesteps=timesteps)
    # Remove the last column (plant identification) for overall evaluation
    x_test_overall = np.delete(dataset["x_test"], -1, axis=2)

    # Initialize dictionaries to hold plant-specific datasets
    plant_specific_x_test = {}
    plant_specific_y_test = {}

    # Get unique plant identifiers
    unique_plants = np.unique(dataset["x_test"][:, 0, -1])

    for plant in unique_plants:
        # Create a mask for the current plant
        plant_mask = dataset["x_test"][:, 0, -1] == plant
        # Filter x_test and y_test using the mask
        x_test_plant = dataset["x_test"][plant_mask]
        y_test_plant = dataset["y_test"][plant_mask]

        # Remove the last column (plant identification) from x_test_plant
        x_test_plant = np.delete(x_test_plant, -1, axis=2)

        # Store the filtered arrays in the dictionaries
        plant_specific_x_test[plant] = x_test_plant
        plant_specific_y_test[plant] = y_test_plant

    # Output the shapes to verify
    # print("Overall x_test shape:", x_test_overall.shape)
    # for plant in plant_specific_x_test:
    #    print(f"Plant {plant} x_test shape:", plant_specific_x_test[plant].shape)
    #    print(f"Plant {plant} y_test shape:", plant_specific_y_test[plant].shape)

    plant_specific = {"x_test": plant_specific_x_test, "y_test": plant_specific_y_test}
    x_test_overall = x_test_overall.astype(float)
    dataset["x_test"] = x_test_overall
    return dataset, plant_specific

<IPython.core.display.Javascript object>

In [29]:
def print_scores_2(scores, METRICS, METRICS_DICT):
    for phase in ["test"]:
        print("******")
        print(f"[{phase.upper()}]")
        print("******")
        for metric in METRICS:
            name = METRICS_DICT[metric]
            print(
                f"{name}: %.3f (%.3f)"
                % (
                    np.mean(scores[f"{phase}_" + metric]),
                    np.std(scores[f"{phase}_" + metric]),
                )
            )
        print("\n======================\n")

<IPython.core.display.Javascript object>

In [30]:
def append_results(scores_dict):
    for plant, scores in scores_dict.items():
        results_dict_copy = results_dict.copy()
        results_dict_copy["Plant"] = plant
        results_dict_copy["Timesteps"] = timesteps
        results_dict_copy["Cross Validation"] = "Out of time"
        results_dict_copy[
            "Cross Validation Params"
        ] = '{"train_size": 0.8, "test_size": 0.2}'
        results_dict_copy["Data Shape"] = x.shape
        results_dict_copy["Model"] = f"Bi-LSTM{model_index}"
        scores = {key: [value] for key, value in scores.items()}
        df_results = fill_results_dict(results_dict_copy, scores)
        results_to_save.append(df_results)


<IPython.core.display.Javascript object>

In [31]:
def print_results_and_get_scores_dict(scores, plant_specific):
    print("=======================")
    print("Overall Score")
    print("=======================\n")

    print_scores(scores, METRICS, METRICS_DICT)

    print("=======================")
    print("Scores per plant")
    print("=======================\n")

    scores_dict = {"INN": scores}

    for plant in df_copy["Factory_Plant"].unique():
        x_test = plant_specific["x_test"][plant].astype(float)
        y_test = plant_specific["y_test"][plant].astype(float)
        dataset["x_test"] = x_test
        dataset["y_test"] = y_test
        print("=======================")
        print(
            "Plant",
            plant,
        )
        print("=======================\n")
        scores_plant = evaluate_model(model, dataset)
        scores_dict[plant] = scores_plant
        print_scores_2(scores_plant, METRICS, METRICS_DICT)
    return scores_dict

<IPython.core.display.Javascript object>

In [32]:
def get_bilstm_params(
    timesteps=1,
    activation="relu",
    callbacks=None,
    validation_split=0.0,
    verbose=0,
):
    params = {}
    params["verbose"] = verbose
    params["callbacks"] = callbacks
    params["validation_split"] = validation_split

    return params

<IPython.core.display.Javascript object>

In [33]:
TIMESTEPS_LIST = [1, 7, 14]

<IPython.core.display.Javascript object>

# Bidirectional LSTM

### BiLSTM 1

1. TIMESTEPS: 1

In [34]:
set_seeds()
set_global_determinism()
timesteps = 1
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM1, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

2024-07-15 22:37:13.426580: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-15 22:37:13.426622: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: inspirada
2024-07-15 22:37:13.426629: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: inspirada
2024-07-15 22:37:13.426792: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.183.1
2024-07-15 22:37:13.426820: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.183.1
2024-07-15 22:37:13.426826: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.183.1


Minutes Elapsed:  0.470673402150472


<IPython.core.display.Javascript object>

In [35]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 4.556 (0.000)
MAE: 3.416 (0.000)
MAPE: 0.079 (0.000)
R2: 0.368 (0.000)


******
[TEST]
******
RMSE: 4.472 (0.000)
MAE: 3.574 (0.000)
MAPE: 0.086 (0.000)
R2: 0.421 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 4.819 (0.000)
MAE: 3.887 (0.000)
MAPE: 0.082 (0.000)
R2: 0.384 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 3.728 (0.000)
MAE: 3.188 (0.000)
MAPE: 0.082 (0.000)
R2: -0.740 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 4.781 (0.000)
MAE: 3.675 (0.000)
MAPE: 0.092 (0.000)
R2: 0.079 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 1

1. TIMESTEPS: 7

In [36]:
set_seeds()
set_global_determinism()
timesteps = 7
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM1, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  1.468625005086263


<IPython.core.display.Javascript object>

In [37]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 5.672 (0.000)
MAE: 4.544 (0.000)
MAPE: 0.100 (0.000)
R2: 0.021 (0.000)


******
[TEST]
******
RMSE: 5.612 (0.000)
MAE: 4.634 (0.000)
MAPE: 0.106 (0.000)
R2: 0.083 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 7.468 (0.000)
MAE: 5.860 (0.000)
MAPE: 0.124 (0.000)
R2: -0.568 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 3.666 (0.000)
MAE: 3.067 (0.000)
MAPE: 0.080 (0.000)
R2: -0.798 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 5.414 (0.000)
MAE: 5.079 (0.000)
MAPE: 0.116 (0.000)
R2: -0.197 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 1

1. TIMESTEPS: 14

In [38]:
set_seeds()
set_global_determinism()
timesteps = 14
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM1, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.7342754364013672


<IPython.core.display.Javascript object>

In [39]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 23.712 (0.000)
MAE: 14.991 (0.000)
MAPE: 0.332 (0.000)
R2: -16.148 (0.000)


******
[TEST]
******
RMSE: 21.214 (0.000)
MAE: 14.440 (0.000)
MAPE: 0.320 (0.000)
R2: -12.266 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 33.481 (0.000)
MAE: 30.077 (0.000)
MAPE: 0.613 (0.000)
R2: -30.265 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 10.799 (0.000)
MAE: 7.059 (0.000)
MAPE: 0.177 (0.000)
R2: -15.211 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 16.207 (0.000)
MAE: 9.747 (0.000)
MAPE: 0.235 (0.000)
R2: -9.826 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 2

1. TIMESTEPS: 1

In [40]:
set_seeds()
set_global_determinism()
timesteps = 1
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM2, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.26414862473805745


<IPython.core.display.Javascript object>

In [41]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 6.756 (0.000)
MAE: 5.327 (0.000)
MAPE: 0.114 (0.000)
R2: -0.389 (0.000)


******
[TEST]
******
RMSE: 6.603 (0.000)
MAE: 5.470 (0.000)
MAPE: 0.121 (0.000)
R2: -0.263 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 9.709 (0.000)
MAE: 8.561 (0.000)
MAPE: 0.170 (0.000)
R2: -1.500 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 3.298 (0.000)
MAE: 2.694 (0.000)
MAPE: 0.070 (0.000)
R2: -0.362 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 5.843 (0.000)
MAE: 5.566 (0.000)
MAPE: 0.128 (0.000)
R2: -0.375 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 2

1. TIMESTEPS: 7

In [42]:
set_seeds()
set_global_determinism()
timesteps = 7
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM2, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.8082012891769409


<IPython.core.display.Javascript object>

In [43]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 9.436 (0.000)
MAE: 6.401 (0.000)
MAPE: 0.138 (0.000)
R2: -1.710 (0.000)


******
[TEST]
******
RMSE: 8.942 (0.000)
MAE: 6.548 (0.000)
MAPE: 0.146 (0.000)
R2: -1.328 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 14.602 (0.000)
MAE: 12.896 (0.000)
MAPE: 0.251 (0.000)
R2: -4.996 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 4.709 (0.000)
MAE: 4.067 (0.000)
MAPE: 0.106 (0.000)
R2: -1.966 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 5.728 (0.000)
MAE: 4.070 (0.000)
MAPE: 0.104 (0.000)
R2: -0.339 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 2

1. TIMESTEPS: 14

In [44]:
set_seeds()
set_global_determinism()
timesteps = 14
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM2, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  2.264544367790222


<IPython.core.display.Javascript object>

In [45]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 16.428 (0.000)
MAE: 11.246 (0.000)
MAPE: 0.242 (0.000)
R2: -7.231 (0.000)


******
[TEST]
******
RMSE: 14.576 (0.000)
MAE: 9.691 (0.000)
MAPE: 0.216 (0.000)
R2: -5.263 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 16.708 (0.000)
MAE: 14.605 (0.000)
MAPE: 0.289 (0.000)
R2: -6.786 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 17.743 (0.000)
MAE: 8.849 (0.000)
MAPE: 0.216 (0.000)
R2: -42.759 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 8.983 (0.000)
MAE: 6.984 (0.000)
MAPE: 0.165 (0.000)
R2: -2.326 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 3

1. TIMESTEPS: 1

In [46]:
set_seeds()
set_global_determinism()
timesteps = 1
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM3, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.7194215019543966


<IPython.core.display.Javascript object>

In [47]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 4.235 (0.000)
MAE: 3.352 (0.000)
MAPE: 0.078 (0.000)
R2: 0.454 (0.000)


******
[TEST]
******
RMSE: 4.268 (0.000)
MAE: 3.716 (0.000)
MAPE: 0.089 (0.000)
R2: 0.472 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 3.598 (0.000)
MAE: 2.905 (0.000)
MAPE: 0.062 (0.000)
R2: 0.657 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 4.170 (0.000)
MAE: 3.681 (0.000)
MAPE: 0.095 (0.000)
R2: -1.177 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 4.780 (0.000)
MAE: 4.349 (0.000)
MAPE: 0.103 (0.000)
R2: 0.080 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 3

1. TIMESTEPS: 7

In [48]:
set_seeds()
set_global_determinism()
timesteps = 7
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM3, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  1.5514445344607035


<IPython.core.display.Javascript object>

In [49]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 5.597 (0.000)
MAE: 4.439 (0.000)
MAPE: 0.097 (0.000)
R2: 0.047 (0.000)


******
[TEST]
******
RMSE: 6.246 (0.000)
MAE: 5.521 (0.000)
MAPE: 0.127 (0.000)
R2: -0.136 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 7.720 (0.000)
MAE: 6.692 (0.000)
MAPE: 0.141 (0.000)
R2: -0.676 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 5.587 (0.000)
MAE: 4.997 (0.000)
MAPE: 0.129 (0.000)
R2: -3.176 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 5.548 (0.000)
MAE: 5.122 (0.000)
MAPE: 0.114 (0.000)
R2: -0.256 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 3

1. TIMESTEPS: 14

In [50]:
set_seeds()
set_global_determinism()
timesteps = 14
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM3, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  1.6005253553390504


<IPython.core.display.Javascript object>

In [51]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 20.072 (0.000)
MAE: 13.113 (0.000)
MAPE: 0.292 (0.000)
R2: -11.288 (0.000)


******
[TEST]
******
RMSE: 15.644 (0.000)
MAE: 11.276 (0.000)
MAPE: 0.254 (0.000)
R2: -6.214 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 23.753 (0.000)
MAE: 19.861 (0.000)
MAPE: 0.394 (0.000)
R2: -14.736 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 13.777 (0.000)
MAE: 10.799 (0.000)
MAPE: 0.274 (0.000)
R2: -25.383 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 8.296 (0.000)
MAE: 5.714 (0.000)
MAPE: 0.141 (0.000)
R2: -1.836 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 4

1. TIMESTEPS: 1

In [52]:
set_seeds()
set_global_determinism()
timesteps = 1
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM4, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.906619401772817


<IPython.core.display.Javascript object>

In [53]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 4.703 (0.000)
MAE: 3.837 (0.000)
MAPE: 0.087 (0.000)
R2: 0.327 (0.000)


******
[TEST]
******
RMSE: 4.359 (0.000)
MAE: 3.704 (0.000)
MAPE: 0.086 (0.000)
R2: 0.450 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 4.156 (0.000)
MAE: 3.396 (0.000)
MAPE: 0.074 (0.000)
R2: 0.542 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 2.829 (0.000)
MAE: 2.291 (0.000)
MAPE: 0.059 (0.000)
R2: -0.002 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 5.455 (0.000)
MAE: 5.150 (0.000)
MAPE: 0.119 (0.000)
R2: -0.198 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 4

1. TIMESTEPS: 7

In [54]:
set_seeds()
set_global_determinism()
timesteps = 7
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM4, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  1.2782010793685914


<IPython.core.display.Javascript object>

In [55]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 17.222 (0.000)
MAE: 14.464 (0.000)
MAPE: 0.323 (0.000)
R2: -8.027 (0.000)


******
[TEST]
******
RMSE: 16.953 (0.000)
MAE: 14.343 (0.000)
MAPE: 0.331 (0.000)
R2: -7.369 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 17.148 (0.000)
MAE: 14.700 (0.000)
MAPE: 0.293 (0.000)
R2: -7.269 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 15.318 (0.000)
MAE: 11.982 (0.000)
MAPE: 0.302 (0.000)
R2: -30.393 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 18.095 (0.000)
MAE: 16.092 (0.000)
MAPE: 0.383 (0.000)
R2: -12.365 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 4

1. TIMESTEPS: 14

In [56]:
set_seeds()
set_global_determinism()
timesteps = 14
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM4, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  1.7202419757843017


<IPython.core.display.Javascript object>

In [57]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 17.979 (0.000)
MAE: 14.291 (0.000)
MAPE: 0.310 (0.000)
R2: -8.858 (0.000)


******
[TEST]
******
RMSE: 16.661 (0.000)
MAE: 13.348 (0.000)
MAPE: 0.301 (0.000)
R2: -7.182 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 21.373 (0.000)
MAE: 18.966 (0.000)
MAPE: 0.378 (0.000)
R2: -11.740 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 15.636 (0.000)
MAE: 13.046 (0.000)
MAPE: 0.326 (0.000)
R2: -32.982 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 13.447 (0.000)
MAE: 9.701 (0.000)
MAPE: 0.228 (0.000)
R2: -6.453 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 5

1. TIMESTEPS: 1

In [58]:
set_seeds()
set_global_determinism()
timesteps = 1
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM5, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  0.289212691783905


<IPython.core.display.Javascript object>

In [59]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 4.818 (0.000)
MAE: 3.578 (0.000)
MAPE: 0.082 (0.000)
R2: 0.294 (0.000)


******
[TEST]
******
RMSE: 4.944 (0.000)
MAE: 3.956 (0.000)
MAPE: 0.093 (0.000)
R2: 0.292 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 5.847 (0.000)
MAE: 4.826 (0.000)
MAPE: 0.098 (0.000)
R2: 0.093 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 4.015 (0.000)
MAE: 3.437 (0.000)
MAPE: 0.089 (0.000)
R2: -1.019 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 4.938 (0.000)
MAE: 3.755 (0.000)
MAPE: 0.094 (0.000)
R2: 0.018 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 5

1. TIMESTEPS: 7

In [60]:
set_seeds()
set_global_determinism()
timesteps = 7
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM5, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  1.5058475136756897


<IPython.core.display.Javascript object>

In [61]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 5.759 (0.000)
MAE: 4.392 (0.000)
MAPE: 0.100 (0.000)
R2: -0.009 (0.000)


******
[TEST]
******
RMSE: 5.108 (0.000)
MAE: 4.349 (0.000)
MAPE: 0.102 (0.000)
R2: 0.240 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 5.946 (0.000)
MAE: 5.137 (0.000)
MAPE: 0.107 (0.000)
R2: 0.006 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 3.603 (0.000)
MAE: 2.954 (0.000)
MAPE: 0.076 (0.000)
R2: -0.736 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 5.526 (0.000)
MAE: 4.966 (0.000)
MAPE: 0.120 (0.000)
R2: -0.247 (0.000)




<IPython.core.display.Javascript object>

### BiLSTM 5

1. TIMESTEPS: 14

In [62]:
set_seeds()
set_global_determinism()
timesteps = 14
model_early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
dataset, plant_specific = prepare_dataset_helper(timesteps=timesteps)
x_train = dataset["x_train"]
y_train = dataset["y_train"]
x_test = dataset["x_test"]
y_test = dataset["y_test"]

params = get_bilstm_params(
    timesteps=timesteps,
    callbacks=model_early_stop,
    validation_split=0.1,
    verbose=0,
)

start = time.time()
model, scores = train_and_evaluate_model(
    BidirectionalLSTM5, dataset, estimator_params=params
)
end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Minutes Elapsed:  1.2871038675308228


<IPython.core.display.Javascript object>

In [63]:
scores_dict = print_results_and_get_scores_dict(scores, plant_specific)
append_results(scores_dict)
model_index += 1

Overall Score

******
[TRAIN]
******
RMSE: 13.664 (0.000)
MAE: 9.749 (0.000)
MAPE: 0.216 (0.000)
R2: -4.694 (0.000)


******
[TEST]
******
RMSE: 12.326 (0.000)
MAE: 10.025 (0.000)
MAPE: 0.229 (0.000)
R2: -3.479 (0.000)


Scores per plant

Plant partner_iv

******
[TEST]
******
RMSE: 15.846 (0.000)
MAE: 13.105 (0.000)
MAPE: 0.263 (0.000)
R2: -6.003 (0.000)


Plant partner_ii

******
[TEST]
******
RMSE: 7.438 (0.000)
MAE: 6.655 (0.000)
MAPE: 0.171 (0.000)
R2: -6.690 (0.000)


Plant partner_i

******
[TEST]
******
RMSE: 12.798 (0.000)
MAE: 10.702 (0.000)
MAPE: 0.255 (0.000)
R2: -5.751 (0.000)




<IPython.core.display.Javascript object>

# Saving the results

In [64]:
path = f"../../../../../../../reports/results/global_models/inn/all_cements/pre_training/full/"
filename = f"BidirectionalLSTM_results_full_{index_to_save}.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [65]:
from sklearn.preprocessing import StandardScaler

<IPython.core.display.Javascript object>

In [66]:
columns_to_standardize = ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]

<IPython.core.display.Javascript object>

In [68]:
ddf = pd.concat(results_to_save).reset_index(drop=True)
ddf_copy = ddf.copy()
ddf_copy = ddf_copy[ddf_copy["Plant"].eq("INN")]
scaler = StandardScaler()
dddf_copy = ddf_copy.copy()
dddf_copy[columns_to_standardize] = scaler.fit_transform(
    ddf_copy[columns_to_standardize]
).copy()

ddf_copy["SCPM"] = (
    dddf_copy[["RMSE Test", "MAE Test", "MAPE Test"]].sum(axis=1) - dddf_copy["R2 Test"]
)
min_row = ddf_copy[ddf_copy["SCPM"].eq(ddf_copy["SCPM"].min())]
ddf.merge(min_row[["Timesteps", "Model"]], on=("Timesteps", "Model"), how="inner")

,Category,Company,Plant,Features,Data Shape,Timesteps,Model,Model Params,Scaler,Scaler Params,...,Cross Validation,Cross Validation Params,RMSE Train,MAE Train,MAPE Train,R2 Train,RMSE Test,MAE Test,MAPE Test,R2 Test
0,Global Model,INN,INN,Chemical + Mineralogical + Properties CS Less,"(3180, 16)",1,Bi-LSTM1,None,Standard Scaler,None,...,Out of time,"{""train_size"": 0.8, ""test_size"": 0.2}",4.556399,3.415909,0.079014,0.368369,4.471897,3.574475,0.085872,0.420796
1,Global Model,INN,partner_iv,Chemical + Mineralogical + Properties CS Less,"(3180, 16)",1,Bi-LSTM1,None,Standard Scaler,None,...,Out of time,"{""train_size"": 0.8, ""test_size"": 0.2}",4.556399,3.415909,0.079014,0.368369,4.819048,3.887414,0.082204,0.384085
2,Global Model,INN,partner_ii,Chemical + Mineralogical + Properties CS Less,"(3180, 16)",1,Bi-LSTM1,None,Standard Scaler,None,...,Out of time,"{""train_size"": 0.8, ""test_size"": 0.2}",4.556399,3.415909,0.079014,0.368369,3.727710,3.187778,0.082313,-0.740116
3,Global Model,INN,partner_i,Chemical + Mineralogical + Properties CS Less,"(3180, 16)",1,Bi-LSTM1,None,Standard Scaler,None,...,Out of time,"{""train_size"": 0.8, ""test_size"": 0.2}",4.556399,3.415909,0.079014,0.368369,4.780912,3.675040,0.091662,0.079461


<IPython.core.display.Javascript object>